<a href="https://colab.research.google.com/github/maddran/lordstanleyscup/blob/main/LSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Clone git repo and get helpers


!rm -r lordstanleyscup/
!git clone https://github.com/maddran/lordstanleyscup

from lordstanleyscup.code.helpers import *

Cloning into 'lordstanleyscup'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 62 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (62/62), done.


In [ ]:
SEASON = 2021 #@param {type:"integer"}
SCWINNER = "TBL" #@param {type:"string"}




In [ ]:
sched_df = get_schedule(SEASON)

draft_data_path = f"/content/lordstanleyscup/draft_data/Stanley Cup - {str(SEASON)[-2:]}_{str(SEASON+1)[-2:]} - Draft.csv"
draft_df = pd.read_csv(draft_data_path)
draft_df.columns = ['round', 'pick', 'owner', 'team']

draft_LU = dict(zip(draft_df.team, draft_df.owner))

In [ ]:
score_df = sched_df

score_df['cupGame'] = 0
score_df['cupTeam'] = None

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def next_cup_game(df, cupteam):
  return min((df[df.cupTeam.isnull()].homeTeam == cupteam).idxmax(),
    (df[df.cupTeam.isnull()].awayTeam == cupteam).idxmax())

def get_winner(row):
  if row.homeScore > row.awayScore:
    return row.homeTeam
  elif row.homeScore < row.awayScore:
    return row.awayTeam
  else:
    return None

def score_season(df, cupteam = SCWINNER):

  if df.cupTeam.last_valid_index() is None:
    last_idx = 0
  else:
    last_idx = df.cupTeam.last_valid_index()
    cupteam = df.loc[last_idx,'cupTeam']

  next_idx = next_cup_game(df, cupteam)
  print(last_idx, next_idx)
  if(last_idx != 0 and next_idx <= last_idx+1):
    return

  df.loc[next_idx, 'cupGame'] = 1
  df.loc[last_idx:next_idx-1, 'cupTeam'] = cupteam
  cupteam = get_winner(df.loc[next_idx])
  df.loc[next_idx, 'cupTeam'] = cupteam

  score_season(df, cupteam)


# Total points and cup games

In [ ]:
points_df = score_df[(score_df.cupGame == 1) & ((score_df.homeScore + score_df.awayScore) != 0)]

points_summary = points_df.groupby(['cupOwner']).cupGame.sum().reset_index()
points_summary.columns = ['owner', 'points']

cup_games_summary_home = points_df.groupby(['homeOwner','homeTeam']).cupGame.sum().reset_index()
cup_games_summary_home.columns = ['owner', 'team', 'cupGames']
cup_games_summary_away = points_df.groupby(['awayOwner','awayTeam']).cupGame.sum().reset_index()
cup_games_summary_away.columns = ['owner', 'team', 'cupGames']

cup_games_summary = pd.concat([cup_games_summary_away, cup_games_summary_home]).groupby(['owner']).cupGames.sum().reset_index()

full_summary = cup_games_summary.merge(points_summary, how = 'outer').fillna(0).sort_values(by=['points', 'cupGames'], ascending=[False,True])
full_summary

,owner,cupGames,points
3,R,44,24
1,E,44,23
2,M,38,21
0,A,32,11


# Points Snake

In [ ]:
snake = points_df.groupby(['gameDate','cupOwner','cupTeam']).cupGame.sum().groupby(level=1).cumsum().reset_index()
snake.columns = ['gameDate', 'owner', 'cupTeam', 'points']
snake = snake.pivot(index='gameDate', columns='owner', values='points')
snake.index = pd.to_datetime(snake.index)
snake.loc[min(snake.index - timedelta(days=1)) -  pd.to_timedelta(1, unit='d')] = 0
snake.iloc[0].fillna(0,inplace=True)
snake.ffill(axis = 0, inplace=True)
snake.sort_index(inplace=True)

snake.tail(5)

owner,A,E,M,R
gameDate,,,,
2022-04-16,10.0,22.0,19.0,24.0
2022-04-18,11.0,22.0,19.0,24.0
2022-04-20,11.0,22.0,20.0,24.0
2022-04-24,11.0,23.0,20.0,24.0
2022-04-26,11.0,23.0,21.0,24.0


In [ ]:
import plotly.express as px

fig = px.line(snake.reset_index(), x='gameDate', y=snake.columns)
fig.update_traces(mode="lines", line_shape="vh", line=dict(width=5))
fig.show()

# Current Cup Team

In [ ]:
currentCupTeam = score_df.loc[score_df.cupTeam.last_valid_index(), 'cupTeam']
currentCupTeam

'ANA'

# Next cup game

In [ ]:
next_cup_idx = next_cup_game(score_df,currentCupTeam)
score_df.loc[next_cup_idx,:]

gameDate     20220427
homeTeam          NYR
awayTeam          MTL
homeScore           3
awayScore           4
cupGame             0
cupTeam          None
homeOwner           R
awayOwner           E
cupOwner             
Name: 1282, dtype: object

# Hypothetical next cup game

In [ ]:
for winner in ['home', 'away']:
  hypo_df = score_df.iloc[next_cup_idx:,:].reset_index(drop=True)
  hypo_df.loc[0, f'{winner}Score'] = 1
  hypo_df.loc[0, 'cupTeam'] = hypo_df.loc[0, f'{winner}Team']
  hypo_df.loc[0, 'cupOwner'] = hypo_df.loc[0, f'{winner}Owner']

  hypo_team = hypo_df.loc[0, 'cupTeam']
  hypo_owner = hypo_df.loc[0, 'cupOwner']
  
  score_season(hypo_df, hypo_team)

  hypo_next_cup_idx = next_cup_game(hypo_df,hypo_team)

  print(f"\nIf {hypo_owner} wins:\n")
  print(hypo_df.loc[hypo_next_cup_idx,:])





0 1
1 2

If R wins:

gameDate     20220427
homeTeam          CHI
awayTeam          VGK
homeScore           4
awayScore           3
cupGame             0
cupTeam          None
homeOwner           E
awayOwner           M
cupOwner             
Name: 2, dtype: object
0 1
1 2

If E wins:

gameDate     20220427
homeTeam          CHI
awayTeam          VGK
homeScore           4
awayScore           3
cupGame             0
cupTeam          None
homeOwner           E
awayOwner           M
cupOwner             
Name: 2, dtype: object
